Панасюк Настя, бкл182

##### 0. О данных

Я решила собрать корпус отзывов на "Портрет Дориана Грея" с livelib. Негативными отзывами я посчитала отзывы с оценкой 0-2 (на livelib в рецензиях обязательно поставить отметку). Обычно такие оценки ставить не принято, поэтому 0-2 значит, что оценка маркированно негативная. Как положительные я хотела брать 4-5, но с 4-ками падала точность, поэтому как положительные я брала 4.5 и 5 (9-10 по десятибальной). <br>
Все-таки с тройками-четверками не до конца понятно, какой шкалы придерживается автор в оценивании, что для него значит 3 или 4.

#### 1. Пишем краулер

###### 1.0 Подготовка

In [4]:
from bs4 import BeautifulSoup
import re
import requests
import random
import time
from fake_useragent import UserAgent
from tqdm.auto import tqdm

In [5]:
session = requests.session()  # чтобы увеличить скорость работы

In [6]:
ua = UserAgent(verify_ssl=False)  # чтобы не заблокировали на сайте

###### 1.1. Собираем ссылки

In [117]:
links = []

In [118]:
def collect_links(page_number):
    for i in range(page_number):
        url = f'https://www.livelib.ru/book/1000003526/reviews-portret-doriana-greya-oskar-uajld/~{i+1}#reviews'
        response = session.get(url, headers={'User-Agent': ua.random})
        page = response.text
        soup = BeautifulSoup(page, 'html.parser')
        reviews_bs = soup.find_all('span', {'itemprop':'url'})
        rev_links = [link.get_text() for link in reviews_bs]
        for link in rev_links:
            links.append(link)
        time.sleep(random.uniform(2.1, 4.2))

In [119]:
collect = collect_links(120)

In [121]:
len(links)

1315

Записываю ссылки в документ, чтобы не перекачивать заново!

In [122]:
with open ('links.txt', 'w', encoding='utf-8') as f:
    for link in links:
        f.write(link + '\n')

На случай, если надо перечитать список ссылок

In [12]:
with open ('links.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    links = [line for line in text.splitlines()]

###### 1.2 Переходим по ссылкам и выкачиваем отзывы

Я решила собрать целый датасет, потому что данные уже скачаны. В будущем, по датасету можно сделать какие-то статистические наблюдения в том числе.<br>
В датасете (.json): <br>
<li>id (номер отзыва на сайте, https://www.livelib.ru/review/1588543 - id = 1588543)</li> 
<li>user (имя пользователя, вдруг, поведение пользователей с никами на латинице отличается от поведения с никами на кириллице)</li>
<li>mark (оценка от 0 до 5)</li>
<li>heading (заголовок отзыва. Тоже может быть полезным, например, какие заголовки у положительных отзывов, а какие у отрицательных)</li>
<li>review (текст отзыва)</li>
<li>likes (количество лайков)</li>
<li>comments (количество комментариев)</li>
Тоже интересно изучить наполнение отзывов (какие они?) с разным кол-вом лайков/комментариев, что оно говорит.

Понимаю, что датасет проблемный, потому что роман спорный сам по себе))) И там слишком много этики и "хороший" и "плохой" употребляется не однозначно в хороших или плохих ревью. Но этим интересно попробовать разобраться именно с ним

###### Разбираем страницу

Возвращаем словарь с данными

In [8]:
def crawldata(rev_id, soup):
    user = soup.find('a', {'class':'header-card-user__name'}).text
    title = soup.find('h3', {'class':'lenta-card__title'}).text
    mark_and_heading = re.findall('(\S+)', title)
    mark = mark_and_heading[0]
    if len(mark_and_heading)>1:
        heading = ' '.join(mark_and_heading[1:])
    else:
        heading = '-'  # если отзыв без заголовка
    review = soup.find('div', {'id':'lenta-card__text-review-full'})
    try:  ## удаляю карточку "с благодарностью" - кто советовал прочитать книгу и советовал ли
        soup.find('div', {'class':'lenta-card__thanks'}).decompose()  # для исследования это не нужная информация
    except:
        pass
    rev_text = review.text.strip(' \n')
    likes = soup.find('span', {'id':f'vote-plus-span-review-{rev_id}'}).text
    comms = soup.find('a', {'class':'sab__link icon-comment'}).text.replace('\n', '')
    return {'id':rev_id, 'user':user, 'mark':mark, 'heading':heading, 'review':rev_text, 'likes':likes, 'comments':comms}

###### Функция для скачивания странички из списка

In [9]:
def processlink(link, batch):
    url = f'https://{link}'
    response = session.get(url, headers={'User-Agent': ua.random})
    response.encoding = 'utf-8'
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    rev_id = link.rsplit('/',1)[1]
    data = crawldata(rev_id, soup)
    batch.append(data)
    time.sleep(random.uniform(2.1, 3.2))         

###### Запуск

На пробу скачиваю первые 500 - все ли работает.

In [21]:
firstbatch = []

In [28]:
for link in tqdm(links[:500]):
    try:
        review = processlink(link, firstbatch)
    except:
        pass

###### 1.3 Подушка безопасности

In [1]:
import json

In [31]:
with open('first500.json', 'w', encoding='utf-8') as f:
    json.dump(firstbatch , f)

In [2]:
with open('first500.json', 'r', encoding='utf-8') as f:
    firstbatch = json.load(f)

500 оказалось недостаточно, было всего 33 отрицательных, докачиваю остальные отзывы

In [10]:
secondbatch = []

In [13]:
for link in tqdm(links[500:]):
    try:
        review = processlink(link, secondbatch)
    except:
        pass

In [15]:
with open('500further.json', 'w', encoding='utf-8') as f:
    json.dump(secondbatch , f)

In [3]:
with open('500further.json', 'r', encoding='utf-8') as f:
    secondbatch = json.load(f)

##### 2. Обрабатываем отзывы

In [4]:
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

Склеиваю выборки

In [69]:
reviews = firstbatch + secondbatch

In [70]:
good = []
bad = []

In [71]:
for item in reviews:  # разделяю отзывы по оценке 0-2 и 4.5-5
    if float(item['mark'])<3:
        bad.append(item['review'])
    elif float(item['mark'])>4:
        good.append(item['review'])

In [72]:
len(good)

650

In [73]:
len(bad)

104

Итак! Однозначно плохих отзывов только 104, тогда как хороших - 650. Давайте тогда возьмем 80 положительных и все не-тестовые отрицательные (отрицательные отзывы сильно короче, поэтому появляются проблемы с неравным количеством слов).<br>К ним 20 на классификацию. Пусть 11 плохих и 9 хороших.

In [230]:
goodsample = good[:80]
badsample = bad[11:]

In [200]:
testsample = []
for item in good[110:119]:
    testsample.append({'text':item, 'gold':'good'})
for item in bad[:11]:
    testsample.append({'text':item, 'gold':'bad'})

###### 2.1 Токенизация через nltk и лемматизация pymorphy

In [63]:
def preproc(sample):
    sampletext=' '.join(sample)
    tokens = [w.lower() for w in word_tokenize(sampletext) if w.isalpha()]
    lemmatized = [morph.parse(token)[0].normal_form for token in tokens]
    return lemmatized

Получаем списки слов для положительных и отрицательных отзывов; делаем сет(множество), потом понадобится для словаря

In [231]:
fromgood = preproc(goodsample)
goodset = set(fromgood)
frombad = preproc(badsample)
badset = set(frombad)

Делаем словари, в итоге имеем отсортированный Counter

In [232]:
good_dict = defaultdict(int)
bad_dict = defaultdict(int)

In [189]:
def filldict(dictionary, lemmatized):
    for word in lemmatized:
        dictionary[word] +=1
    return dictionary

In [233]:
good_counted = Counter(filldict(good_dict, fromgood)).most_common()
bad_counted = Counter(filldict(bad_dict, frombad)).most_common()

###### 2.2 Получаем списки слов, характерных только для "плохих" и "хороших" отзывов; выкидываем единичные употребления

<i>Вопрос</i>: Как выделит уникальные для множества слова, которые не встречаются в другом, умнее, чем setA - setB?

In [234]:
chosengood = list(goodset-badset)
chosenbad = list(badset-goodset)

In [134]:
def leavespecified(counteddict, comparisonset):
    workset = []
    for item in counteddict:
        if item[0] in comparisonset and item[1]>1:  # убираю "шум" - слова, которые встретились меньше одного раза
            workset.append(item[0])
    return workset

In [237]:
specifiedgood = leavespecified(good_counted, chosengood)

In [238]:
len(specifiedgood)

409

Как видно, слова скорее не "положительные", а более чем случайные. Но есть положительные оценки типа "мастерски", "необыкновенно", "восхитить", "изящество", "гармонировать".

In [239]:
specifiedgood

['нести',
 'счастливый',
 'отдать',
 'роль',
 'чистота',
 'чуть',
 'точно',
 'выбор',
 'менее',
 'душевный',
 'обстановка',
 'опыт',
 'осознать',
 'глава',
 'достаточно',
 'роковой',
 'стараться',
 'источник',
 'эгоизм',
 'соблазн',
 'пугать',
 'сохранить',
 'жертва',
 'неужели',
 'полка',
 'цель',
 'облик',
 'губительный',
 'xix',
 'добраться',
 'ценный',
 'собственно',
 'интересовать',
 'развитие',
 'насыщенный',
 'приём',
 'ведомый',
 'богатый',
 'чужой',
 'ничуть',
 'окружение',
 'удаться',
 'одинаково',
 'безразличный',
 'летний',
 'жажда',
 'крайне',
 'превращаться',
 'воспоминание',
 'рисовать',
 'невинность',
 'актуально',
 'покорить',
 'несмотря',
 'привыкнуть',
 'обязательно',
 'купить',
 'велик',
 'центральный',
 'эстетика',
 'шок',
 'помощь',
 'причина',
 'оболочка',
 'навсегда',
 'порядок',
 'праздный',
 'необыкновенно',
 'выделить',
 'доля',
 'отступление',
 'знакомиться',
 'чувствоваться',
 'интеллект',
 'выписывать',
 'гармония',
 'преследовать',
 'полезный',
 'оторвать

In [240]:
specifiedbad = leavespecified(bad_counted, chosenbad)

Здесь уже более оформленные отрицательные оценки - "чёртов", "пошлый", "пустышка", "идиот".

In [241]:
specifiedbad

['убить',
 'дуже',
 'вышить',
 'долго',
 'золотой',
 'нож',
 'нечего',
 'отзыв',
 'симпатия',
 'зато',
 'явно',
 'претензия',
 'і',
 'называть',
 'наслаждаться',
 'поклонник',
 'парень',
 'наследство',
 'переехать',
 'новинка',
 'мягко',
 'губить',
 'противоречить',
 'неприятно',
 'як',
 'куда',
 'восхвалять',
 'целый',
 'хватать',
 'тянуться',
 'загадка',
 'смешной',
 'осилить',
 'реальность',
 'пропитать',
 'противный',
 'd',
 'спасти',
 'шёлк',
 'серебряный',
 'парча',
 'занять',
 'статус',
 'нарциссизм',
 'дневник',
 'куча',
 'личико',
 'дійсный',
 'що',
 'мені',
 'мена',
 'він',
 'сотня',
 'клуб',
 'бла',
 'описываться',
 'признанный',
 'вода',
 'мучительный',
 'признавать',
 'любопытный',
 'увлечься',
 'дивный',
 'раздражать',
 'нисколько',
 'плоский',
 'повезти',
 'манер',
 'великий',
 'сборник',
 'встречать',
 'равнодушие',
 'объяснить',
 'потратить',
 'резко',
 'гомосексуализм',
 'тапка',
 'неловко',
 'низкий',
 'нагромождение',
 'мода',
 'проснуться',
 'оказывать',
 'оба',
 '

In [252]:
len(specifiedbad)

294

###### 2.3 Функция, которые определяет положительность и отрицательность

Решила оставить препроцессинг каждого отзыва внутри функции, чтобы в функцию можно было подать любой необработанный текст (а не уже лемматизированные списки).

In [216]:
def predictsentiment(review, goodlist, badlist):
    lemmatized = preproc(review)
    good, bad = 0, 0
    for word in lemmatized:
        if word in goodlist:
            good +=1
        elif word in badlist:
            bad +=1
    if good>bad:
        return 'good'
    elif bad>good:
        return 'bad'
    else:
        return '-'  # если одинаковое количество, прочерк

In [243]:
testfirst = predictsentiment(testsample[0]['text'], specifiedgood, specifiedbad)
print('Предсказание:', testfirst)
print('На самом деле:', testsample[0]['gold'])

Предсказание: good
На самом деле: good


In [244]:
print(testsample[0]['text'])

Необыкновенная книга, со своей, конечно, моралью.
Мне очень понравилось как автор описал личностные качества и портрет главного героя, и в целом мне нравится как пишет автор
Сюжет мистически необыкновенен и концовка нестандартнаяя


In [245]:
testfirst = predictsentiment(testsample[7]['text'], specifiedgood, specifiedbad)
print('Предсказание:', testfirst)
print('На самом деле:', testsample[7]['gold'])

Предсказание: bad
На самом деле: good


In [246]:
print(testsample[7]['text'])

Dolce Vita, или Уродство души
"Манифест европейского эстетизма и шедевр английской художественной прозы" (по мнению Хорхе Луиса Борхеса) стал вкусной вишней на моём книжном апрельском торте.
Роскошный язык, нетривиальный сюжет, нужная толика мистицизма и гедонизма, порция мужского цинизма и страшная трагедия одной человеческой души на фоне общей деградации. Всё, как я люблю!
Дориан, по моему мнению, глубоко несчастен не потому, что в нем так много порока, а потому, что никто ему об этом так и не сказал. Была, конечно, попытка у Бэзила. Но самые страшные для нашей совести вещи нам должны говорить те, кто представляется авторитетом. Иначе любая попытка открыть глаза на нашу безнравственность обречена утонуть в волнах негодования. Гарри - единственный, кто мог бы. Но именно ему принадлежит сомнительная заслуга воспитания Дориана в традициях нарциссизма и вакханалий. Да и с Бэзила нельзя снимать долю вины в нравственном падении Грея.
Иными словами, Дориан Грей - плод селекции всех и вся, п

In [250]:
testfirst = predictsentiment(testsample[9]['text'], specifiedgood, specifiedbad)
print('Предсказание:', testfirst)
print('На самом деле:', testsample[9]['gold'])

Предсказание: good
На самом деле: bad


In [249]:
print(testsample[9]['text'])

Книга, которая оставила меня равнодушной и не произвела впечатления..История Дориана Грея знакома каждому. Многие читали данный роман, а многие смотрели экранизацию. Сюжет в данном случае можно описать очень кратко и незамысловато: в один прекрасный день талантливый художник Бэззил Холлуорд пишет портрет Дориана Грея. Бэззил вкладывает в него все свои эмоции, свой талант и всю душу. И в итоге.. мы "видим" лучшую работу мастера за все его годы! Главный герой, взглянув на портрет, осознаёт насколько он красив и прекрасен. Он восхищён, но в то же время и сильно разочарован. Дориан понимает, что портрет навсегда останется таким прекрасным, а его телесная, физическая оболочка будет без конца увядать с каждым днём.. И в сердцах он проговаривает, что хотел бы навсегда остаться молодым.. А все невзгоды, возрастные изменения перенести на портрет.. Так называемая "сделка с дьяволом".. Интересно, не правда ли? Да и кто бы отказался оттого, чтобы навсегда быть молодым и красивым? С романом у меня 

###### 2.4 Считаем accuracy

In [55]:
from sklearn.metrics import accuracy_score

In [225]:
def test_simple_sentiment_predict(testsample, goodlist, badlist):
    results = []
    gold = []
    for item in testsample:
        prediction = predictsentiment(item['text'], goodlist, badlist)
        results.append(prediction)
        gold.append(item['gold'])
    print(results)
    print("RESULTS:")
    print("%d reviews" % len(testsample))
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [251]:
testwithk = test_simple_sentiment_predict(testsample, specifiedgood, specifiedbad)

['good', 'good', '-', 'good', 'good', 'good', 'good', 'bad', 'good', 'good', 'bad', 'good', 'bad', 'good', 'good', 'good', 'bad', 'good', 'bad', 'good']
RESULTS:
20 reviews
Accuracy: 0.5500


Распечатала еще вывод, чтобы было видно, где классификатор ошибся. Первые девять на самом деле хорошие, остальные - плохие. Как видно из примера с текстом 9. В отзывах слишком перемешано "прекрасное" и "отвратительное" - но такой сам роман.

###### Улучшения

<li>Коэффициент, который будет "скрадывать" разницу в количестве положительных и отрицательных отзывов.<br>Я пробовала k = len(specifiedgood) / len(specifiedbad)</li> <br>
<li>С такой противорчевой и самой по себе "аморальной" книгой должны быть более совершенные методы оценивания положительности/отрицательности, чем проверка по словам. Компьютер должен понимать смысл и сферы действия.</li><br>

<b>Как доп. идеи:</b><br>
<li>Зависимость оценки от заголовка</li>
<li>Соотношение лайков, комментариев и оценки</li><br>
<li>"Выкачать" корпус отзывов на классику, на livelib "Портрет" 12-ый в топе классики. Посмотреть на отзывы повнимательнее.</li>
<li>Сделать "нормирование" оценок по каждому пользователю. Понять, какая у него шкала оценивания, у кого 4 значит "мне не совсем понравилось, но неплохо", а у кого 3 и 4 - "хорошо, но не дотянуло до шедевра" (а 5-ки они вообще в иключительных случаях ставят).</li>
<li>После "нормирования" шкал можно получить взвешенную оценку книги.</li>